importing libraries

In [ ]:
import pandas as pd

Loading the dataset

In [ ]:
df = pd.read_csv("bertinput.csv")

In [ ]:
df.shape

(112011, 4)

In [ ]:
df.isnull().sum()

question_title        250
question_content    38422
answer               2317
grade                   0
dtype: int64

In [ ]:
df.dropna(inplace = True)

In [ ]:
df.head()

,question_title,question_content,answer,grade
0,please review assignment,dear someone please review assignment could al...,also could please check link provide seem damage,0.861111
1,please review assignment,dear someone please review assignment could al...,yeah review assignment checked perhaps grade a...,0.861111
2,please review assignment,dear someone please review assignment could al...,glad could provide link assignment thanks lot ...,0.861111
3,please review assignment,dear someone please review assignment could al...,thanks lot review assignment review,0.861111
4,please review assignment,dear someone please review assignment could al...,hey glad review could please review mine well ...,0.861111


Sentiment analysis using BERT

In [ ]:
pip install transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
model.eval()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = logits.softmax(dim=1)
        sentiment_score = probs[0][1].item()  # Assuming 1 corresponds to positive sentiment
        return sentiment_score


In [ ]:
subset_df = df.sample(n=500, random_state=42)

In [ ]:
subset_df.isnull().sum()

question_title      0
question_content    0
answer              0
grade               0
dtype: int64

In [ ]:
subset_df['Question_Sentiment'] = subset_df['question_content'].apply(predict_sentiment)


In [ ]:
subset_df['Answer_Sentiment'] = subset_df['answer'].apply(predict_sentiment)

In [ ]:
subset_df.head(30)

,question_title,question_content,answer,grade,Question_Sentiment,Answer_Sentiment
29279,please review assignment,hi anyone still please spare time review assig...,review please review mine,0.561111,0.661431,0.660488
69342,please review assignment,assignment long time pls review,good,0.797222,0.665953,0.669550
18063,please review assignment,hi chance please review complete assignment n n,hi saketh review assignment paula,0.972222,0.686369,0.677192
48787,please review assignment,heyy someone please review assignment wait som...,please review,0.994444,0.686083,0.667364
14699,please review assignment,hey everyone review assignment gladly review p...,mine please,0.883333,0.652109,0.704087
32861,please review assignment,hey please review assignment pending link advance,n n please review also,0.452778,0.646112,0.664343
50305,please review assignment,would grateful anybody could take look assignm...,hi review great btw could review mine mind thanks,0.838889,0.655791,0.674549
100805,quiz,get access quiz,hi bhanuka need contact coursera help center d...,0.808333,0.635058,0.678446
1383,please review assignment,hello everyone please review assignment time a...,already review assignment would ngrateful coul...,0.761111,0.662676,0.683745
50784,please review assignment,hi everyone please could take time review assi...,sure thank,0.875000,0.679747,0.654594


In [ ]:
subset_df.corr()

<ipython-input-19-f41d0418d4c5>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  subset_df.corr()


,grade,Question_Sentiment,Answer_Sentiment
grade,1.000000,0.157716,-0.080579
Question_Sentiment,0.157716,1.000000,-0.097251
Answer_Sentiment,-0.080579,-0.097251,1.000000


Feature Extraction using bert

In [ ]:
from transformers import BertTokenizer, BertModel
import torch


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()  #


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:

# Create an empty list to store the BERT embeddings
bert_embeddings = []

# Iterate through each row in the DataFrame
for index, row in subset_df.iterrows():
    # Concatenate the 'question_title', 'question_content', and 'answer' columns
    text = row['question_title'] + ' ' + row['question_content'] + ' ' + row['answer']

    # Tokenize the text and convert it to BERT embeddings
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state.mean(dim=1)  # Average the embeddings across all tokens
        bert_embeddings.append(embeddings)

# Convert the list of BERT embeddings to a DataFrame
bert_embeddings_df = pd.DataFrame(torch.cat(bert_embeddings).numpy())

# Concatenate the BERT embeddings DataFrame with the original DataFrame
result_df = pd.concat([df, bert_embeddings_df], axis=1)




In [ ]:
result_df.isnull().sum()

question_title      0
question_content    0
answer              0
grade               0
0                   0
                   ..
763                 0
764                 0
765                 0
766                 0
767                 0
Length: 772, dtype: int64

In [ ]:
result_df.dropna(inplace= True)

In [ ]:
result_df.head()

,question_title,question_content,answer,grade,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,please review assignment,dear someone please review assignment could al...,also could please check link provide seem damage,0.861111,0.211558,-0.000285,0.463752,0.002897,0.114620,-0.456996,...,0.424881,-0.080847,-0.293234,-0.149905,0.084517,-0.089919,-0.450198,-0.044058,-0.187655,-0.211452
1,please review assignment,dear someone please review assignment could al...,yeah review assignment checked perhaps grade a...,0.861111,-0.078974,-0.070287,0.164930,0.085294,0.093125,-0.292054,...,0.371546,-0.237413,0.178436,-0.236941,-0.176354,-0.020680,-0.310162,-0.138982,-0.054837,0.024507
2,please review assignment,dear someone please review assignment could al...,glad could provide link assignment thanks lot ...,0.861111,-0.086046,-0.008906,0.366793,0.011230,-0.001572,-0.181883,...,0.046385,-0.065006,-0.127343,-0.025654,-0.119414,0.030480,-0.247958,-0.124945,-0.049228,-0.036783
3,please review assignment,dear someone please review assignment could al...,thanks lot review assignment review,0.861111,0.221717,0.182200,0.693043,-0.126354,0.177964,-0.379516,...,0.048652,-0.134217,0.092305,0.048658,-0.179255,-0.125334,-0.186899,-0.314749,-0.152709,-0.045591
4,please review assignment,dear someone please review assignment could al...,hey glad review could please review mine well ...,0.861111,0.226730,0.382649,0.678196,-0.029209,0.166483,-0.224788,...,0.212152,0.083012,-0.197447,-0.156456,-0.044797,-0.150559,-0.295737,0.074154,-0.175765,-0.059989


assigning values in variables

In [ ]:
x_df = result_df

In [ ]:

# List of column names to drop
columns_to_drop = ['question_title', 'question_content', 'answer','grade']

# Drop the specified columns from the DataFrame
x_df = x_df.drop(columns=columns_to_drop)




In [ ]:
X= x_df.values

In [ ]:
y= result_df['grade'].values

Splitting the data

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)



Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)

X_test= scaler.transform(X_test)


Linear Regression

In [ ]:

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score



model = LinearRegression()

model.fit(X_train, y_train)


y_pred = model.predict(X_test)


mae = mean_absolute_error(y_test, y_pred)


r2 = r2_score(y_test, y_pred)

print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)



Mean Absolute Error (MAE): 0.0018038769315978007
R-squared (R2) Score: -4.977266262687554e+25


Decision Tree Regressor

In [ ]:
impo

In [ ]:

from sklearn.model_selection import cross_val_score
import numpy as np

for depth in [1,2,3,4,5,6,7,8,9,10,20]:
  dt = DecisionTreeRegressor(max_depth=depth)

  dt.fit(X_train, y_train)

  dt = DecisionTreeRegressor(max_depth=depth) # a fresh model which is not trained yet
  valAccuracy = cross_val_score(dt, X_train, y_train, cv=10) # syntax : cross_val_Score(freshModel,fts, target, cv= 10/5)
  print("Depth  : ", depth,  " Cross val score : " ,np.mean(valAccuracy))




Depth  :  1  Cross val score :  -16.678378378378376
Depth  :  2  Cross val score :  -187.73243243243246
Depth  :  3  Cross val score :  -209.23193456614513
Depth  :  4  Cross val score :  -206.30170697012804
Depth  :  5  Cross val score :  -196.3839971550498
Depth  :  6  Cross val score :  -198.09039829302986
Depth  :  7  Cross val score :  -202.74935988620197
Depth  :  8  Cross val score :  -191.06657183499289
Depth  :  9  Cross val score :  -201.59608819345658
Depth  :  10  Cross val score :  -204.55035561877668
Depth  :  20  Cross val score :  -203.92603129445234


KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
import numpy as np

# Define a list of values for k
k_values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50]

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)  # Initialize the KNN regressor with the current k value
    knn.fit(X_train, y_train)  # Train the KNN model on the training data

    # Calculate the R-squared score on the training data
    train_score = knn.score(X_train, y_train)

    # Calculate the cross-validation score using 10-fold cross-validation
    cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
    cv_score_mean = np.mean(cv_scores)

    print(f"K value: {k}, Train Score: {train_score:.4f}, CV Score (mean): {cv_score_mean:.4f}")


K value: 1, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 2, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 3, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 4, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 5, Train Score: 0.7500, CV Score (mean): 0.0000
K value: 6, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 7, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 8, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 9, Train Score: 1.0000, CV Score (mean): 1.0000
K value: 10, Train Score: 0.7500, CV Score (mean): 0.0000
K value: 20, Train Score: 0.7500, CV Score (mean): 0.0000
K value: 50, Train Score: 0.7500, CV Score (mean): 0.0000


In [ ]:

from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, r2_score




# Create a KNN Regressor model with k=5 (you can change k as needed)
k = 5
model = KNeighborsRegressor(n_neighbors=k)

# Train the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate R-squared (R2) score
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("R-squared (R2) Score:", r2)


Mean Absolute Error (MAE): 1.1102230246251565e-16
R-squared (R2) Score: 0.8888888888888888


XGBoost

In [ ]:

import xgboost as xgb

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Initialize and train the XGBoost regressor
xgb_regressor = xgb.XGBRegressor(
    objective='reg:squarederror',  # Use squared error for regression
    n_estimators=100,             # Number of boosting rounds
    learning_rate=0.1,            # Learning rate

    random_state=42               # Random seed for reproducibility
)

xgb_regressor.fit(X_train, y_train)

# Make predictions on the test set
y_pred = xgb_regressor.predict(X_test)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2) Score: {r2}")

# Optionally, you can save the trained model for future use
# xgb_regressor.save_model('xgboost_grade_predictor.model')


Mean Absolute Error (MAE): 9.893882659928188e-06
Root Mean Squared Error (RMSE): 9.893882659928188e-06
R-squared (R2) Score: -8.824102449769596e+20


Neural Netrwork Model

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have already defined X_train, X_test, y_train, and y_test as NumPy arrays

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define a custom neural network model
class GradePredictor(nn.Module):
    def __init__(self, input_size):
        super(GradePredictor, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x

# Initialize the model and optimizer
model = GradePredictor(input_size=X_train_tensor.shape[1])
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()  # Mean Squared



In [ ]:
# Assuming you have defined X_train_tensor, y_train_tensor, model, optimizer, and criterion as mentioned earlier

# Set the number of training epochs
num_epochs = 10  # You can adjust this as needed

# Training loop
for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()

    # Forward pass
    outputs = model(X_train_tensor)

    # Compute the loss
    loss = criterion(outputs, y_train_tensor)

    # Backpropagation and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Training finished.')


Epoch [1/10], Loss: 0.8759
Epoch [2/10], Loss: 0.6923
Epoch [3/10], Loss: 0.5411
Epoch [4/10], Loss: 0.4076
Epoch [5/10], Loss: 0.2908
Epoch [6/10], Loss: 0.1955
Epoch [7/10], Loss: 0.1267
Epoch [8/10], Loss: 0.0872
Epoch [9/10], Loss: 0.0752
Epoch [10/10], Loss: 0.0818
Training finished.


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([371])) that is different to the input size (torch.Size([371, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [ ]:
# Set the model to evaluation mode
model.eval()

# Disable gradient computation for evaluation
with torch.no_grad():
    # Forward pass on the test data
    test_outputs = model(X_test_tensor)

# Convert the predictions back to NumPy arrays
y_pred = test_outputs.numpy()

# Calculate evaluation metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f'Mean Absolute Error: {mae:.4f}')
print(f'Mean Squared Error: {mse:.4f}')
print(f'R-squared (R2) Score: {r2:.4f}')


Mean Absolute Error: 0.2542
Mean Squared Error: 0.1095
R-squared (R2) Score: -986647767771897559866333462528.0000
